<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/Transformer_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [90]:
class CustomDataset(Dataset):
  def __init__(self, train=True):
    self.root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"
    self.url = self.root_url + ("FordA_TRAIN.tsv" if train else "FordA_TEST.tsv")
    self.data = np.loadtxt(self.url, delimiter="\t")
    self.x = self.data[:, 1:]
    self.y = self.data[:, 0].astype(int)
    self.y[self.y == -1] = 0
  
  def __len__(self,):
    return len(self.data)
  
  def __getitem__(self, idx):
    inputs = self.x[idx]
    labels = self.y[idx]
    return (inputs, labels)

  def collate_fn(self, batch):
    inputs = []
    labels = []
    for sample in batch:
      inputs.append(sample[0])
      labels.append(sample[1])
    inputs = torch.tensor(inputs)
    labels = torch.tensor(labels, dtype=torch.long)
    return {
        'inputs': inputs[:, :, None],
        'labels': labels[:, None, None],
    }

In [93]:
class PreNorm(nn.Module):
  def __init__(self, dim, fn):
    super().__init__()
    self.layernorm = nn.LayerNorm(dim)
    self.fn = fn
  
  def forward(self, x, **kwargs):
    return self.fn(self.layernorm(x), **kwargs)

In [94]:
class Residual(nn.Module):
  def __init__(self, fn):
    super().__init__()
    self.fn = fn
  
  def forward(self, x, **kwargs):
    return self.fn(x, **kwargs) + x

In [95]:
class FeedForward(nn.Module):
  def __init__(self, dim, hidden_dim, dropout=0.):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(dim, hidden_dim),
        nn.GELU(),
        nn.Dropout(dropout),
        nn.Linear(hidden_dim, dim),
        nn.Dropout(dropout),
    )
  
  def forward(self, x):
    return self.net(x)